In [19]:
import os
import re
import fitz  # PyMuPDF
import json

# To get the full path to the PDF file
path = os.path.join(os.getcwd(), "hcidata.pdf")
pdf = fitz.open(path)

all_reports = []

for page_num in range(len(pdf)):
    page = pdf[page_num]
    page_text = page.get_text("text")
    
    pattern = r"(REPORT\nID: \w+_\d+)"
    reports = re.split(pattern, page_text)
    reports = reports[1:]  # Skip the first match which is always an empty string

    for report in zip(reports[0::2], reports[1::2]):
        details = {}
        details['report_id'] = re.search(r"ID: (.+)", report[0]).group(1).strip()
        details['report_date'] = re.search(r"REPORTDATE: (.+)", report[1]).group(1).strip()
        # Extract report_description in multiple lines
        details['report_description'] = re.findall(r"REPORTDESCRIPTION: ([\s\S]+?)(?=PERSONS:|$)", report[1])
        details['persons'] = re.findall(r"PERSONS: ([\s\S]+?)(?=DATES:|$)", report[1])
        details['dates'] = re.findall(r"DATES: ([\s\S]+?)(?=PLACES:|$)", report[1])
        details['places'] = re.findall(r"PLACES: ([\s\S]+?)(?=ORGANIZATIONS:|$)", report[1])
        details['organizations'] = re.findall(r"ORGANIZATIONS: ([\s\S]+?)(?=\n\n|$)", report[1])
        all_reports.append(details)

# Close the PDF after processing
pdf.close()


# Save the extracted report details to a JSON file
output_file = "processed_data.json"
with open(output_file, "w") as json_file:
    json.dump(all_reports, json_file, indent=4)